In [1]:
import pandas as pd
import swifter

info = "info_CUB.csv"
df = pd.read_csv(info, sep=";")

def read_text(path):
    with open(path, "r") as f:
        text = f.read()
    return text

df["text"] = df["text"].swifter.apply(read_text)
# Get lenght words in each text
df["n_words"] = df["text"].swifter.apply(lambda x: len(x.split()))

# Convert labels (str) to int
df["label"] = df["label"].astype("category").cat.codes

labels = df["label"].unique()

print(df["n_words"].describe())
df

In [ ]:
columns = ["text", "label"]
data_train = df[df["train"] == 1][columns]
data_test = df[df["train"] == 0][columns]

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(data_train).remove_columns(["__index_level_0__"])
test_dataset = Dataset.from_pandas(data_test).remove_columns(["__index_level_0__"])

In [ ]:
# Split train in train and validation
train_dataset = train_dataset.train_test_split(test_size=0.1)
eval_dataset = train_dataset["test"]
train_dataset = train_dataset["train"]


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2", padding="max_length", truncation=True)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

loading file vocab.txt from cache at /home/jlortiz/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/vocab.txt
loading file tokenizer.json from cache at /home/jlortiz/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/jlortiz/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/jlortiz/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/tokenizer_config.json


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("sentence-transformers/all-mpnet-base-v2", num_labels=len(labels))#.to("cuda")

loading configuration file config.json from cache at /home/jlortiz/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/config.json
Model config MPNetConfig {
  "_name_or_path": "sentence-transformers/all-mpnet-base-v2",
  "architectures": [
    "MPNetForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
 

In [ ]:
import numpy as np
from transformers import TrainingArguments
from transformers import Trainer

import evaluate
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="steps", 
                                  save_strategy="steps",
                                  eval_steps=200,
                                  save_steps=200,
                                  load_best_model_at_end=True,
                                  learning_rate=2e-5, 
                                  per_device_train_batch_size=32, 
                                  per_device_eval_batch_size=32, 
                                  num_train_epochs=50, 
                                  weight_decay=0.01, 
                                  fp16=True)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: text. If text are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5371
  Num Epochs = 50
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 2100
  Number of trainable parameters = 109640264


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/models/mpnet/modeling_mpnet.py", line 726, in forward
    outputs = self.mpnet(
  File "/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1190, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/models/mpnet/modeling_mpnet.py", line 554, in forward
    encoder_outputs = self.encoder(
  File "/home/jlortiz/anaconda3/envs/py310/l

In [ ]:
trainer.evaluate(test_dataset)

The following columns in the evaluation set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: text. If text are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5820
  Batch size = 32


{'eval_loss': 4.888920783996582,
 'eval_accuracy': 0.10584192439862543,
 'eval_runtime': 52.1403,
 'eval_samples_per_second': 111.622,
 'eval_steps_per_second': 3.491,
 'epoch': 4.0}

In [ ]:
# Predict test set with the model
from sklearn.metrics import classification_report
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)
print(classification_report(test_dataset["label"], predictions))


The following columns in the test set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: text. If text are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5820
  Batch size = 32
/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        29
           1       0.00      0.00      0.00        34
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00        29
           4       0.00      0.00      0.00        24
           5       0.00      0.00      0.00        20
           6       0.04      0.04      0.04        25
           7       0.17      0.04      0.06        25
           8       0.00      0.00      0.00        28
           9       0.62      0.70      0.66        30
          10       0.00      0.00      0.00        30
          11       0.40      0.08      0.13        26
          12       0.00      0.00      0.00        30
          13       0.22      0.97      0.35        32
          14       0.13      0.39      0.19        28
          15       0.20      0.93      0.34        28
          16       0.00      0.00      0.00        29
          17       0.00    

/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r